<br>
<h1 style = "font-size:40px; font-family:Garamond ; font-weight : normal; background-color: #C66363 ; color : #E8D6D8; text-align: center; border-radius: 100px 100px;">LEAK DATA </h1>
<br>


# Add Libaries

In [ ]:
import numpy as np # linear algebra
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense , Flatten
from keras.layers import LSTM
from keras.layers import Dropout
from xgboost import XGBRegressor
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
import os
import warnings
warnings.filterwarnings("ignore")

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Load and Examine Data

In [ ]:
train_df = pd.read_csv("../input/tabular-playground-series-jul-2021/train.csv") #loading datas
test_df = pd.read_csv("../input/tabular-playground-series-jul-2021/test.csv")

In [ ]:
train_targets_df = train_df.loc[:,["target_carbon_monoxide","target_benzene","target_nitrogen_oxides"]]
train_df = train_df.drop(["target_carbon_monoxide","target_benzene","target_nitrogen_oxides"],axis = 1)

In [ ]:
len(test_df)

In [ ]:
len(train_df)

In [ ]:
train_df.head()

In [ ]:
train_targets_df.head()

In [ ]:
train_data_len = 7111

# Concatenate Data

In [ ]:
train_df = pd.concat([train_df,test_df],axis = 0).reset_index(drop = True)

In [ ]:
pred_times=train_df[["date_time"]]
train_df=train_df.drop(["date_time"],axis = 1)
pred_times=pred_times.iloc[7111:,:]

In [ ]:
pred_times.reset_index(drop=True, inplace=True)

In [ ]:
scaler = StandardScaler()

train_df = scaler.fit_transform(train_df)

In [ ]:
train_df = pd.DataFrame(train_df)

In [ ]:
test_df = train_df[7111:]
train_df = train_df[:7111]

# Make Models

In [ ]:
"""def get_lined_data(data_x , data_y , data_len , time_len=10):
    x = []
    y = []
    pre_con = pd.DataFrame(columns = ["deg_C","relative_humidity","absolute_humidity","sensor_1","sensor_2","sensor_3","sensor_4","sensor_5"])
    rep_lay = data_x.iloc[[0]]
    
    for i in range(time_len-1):
        pre_con = pre_con.append(rep_lay)
    
    pre_con=pre_con.reset_index(drop=True)
    
    X_con_train = pd.concat([pre_con, data_x], axis=0).reset_index(drop=True)
    X_con_train=X_con_train.to_numpy()
    data_y=data_y.to_numpy()

    for i in range(data_len):
    
        x.append(X_con_train[i:i+time_len,:])
        y.append(data_y[i+time_len-1,:])
        
    x, y = np.array(x), np.array(y)
    
    return x, y"""

### Prepare Inputs

In [ ]:
x = []
y = []
pre_con = pd.DataFrame(columns = [0,1,2,3,4,5,6,7])
rep_lay = train_df.iloc[[0]]

for i in range(59):
    pre_con = pre_con.append(rep_lay)
pre_con=pre_con.reset_index(drop=True)

X_con_train = pd.concat([pre_con, train_df], axis=0).reset_index(drop=True)
X_con_train=X_con_train.to_numpy()
train_targets_df=train_targets_df.to_numpy()

for i in range(7111):
    
    x.append(X_con_train[i:i+60,:])
    y.append(train_targets_df[i,:])
        
x, y = np.array(x), np.array(y)


In [ ]:
x_test = []

pre_con_test = pd.DataFrame(columns = [0,1,2,3,4,5,6,7])
rep_lay_test = test_df.iloc[[0]]

for i in range(59):
    pre_con_test = pre_con_test.append(rep_lay_test)
pre_con_test=pre_con_test.reset_index(drop=True)

X_con_test = pd.concat([pre_con_test, test_df], axis=0).reset_index(drop=True)
X_con_test=X_con_test.to_numpy()

for i in range(2247):
    
    x_test.append(X_con_test[i:i+60,:])
        
x_test= np.array(x_test)

In [ ]:
x.shape

In [ ]:
y_car=y[:,0]
y_car=y_car.reshape(-1,1)

y_benz=y[:,1]
y_benz=y_benz.reshape(-1,1)

y_nit=y[:,2]
y_nit=y_nit.reshape(-1,1)

In [ ]:
y_car.shape

In [ ]:
regressor_car = Sequential()
regressor_car.add(LSTM(units = 90, return_sequences = True, input_shape = (x.shape[1], x.shape[2])))
regressor_car.add(Dropout(0.2))
regressor_car.add(LSTM(units = 50, return_sequences = True))
regressor_car.add(Dropout(0.2))
regressor_car.add(LSTM(units = 50, return_sequences = True))
regressor_car.add(Dropout(0.2))
regressor_car.add(LSTM(units = 50))
regressor_car.add(Dropout(0.2))
regressor_car.add(Dense(units = 1))

regressor_car.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
history = regressor_car.fit(x, y_car, epochs = 110, batch_size = 32)

In [ ]:
def display_learning_curves(history):

    plt.plot(history.history["loss"])
    plt.legend(["train"], loc="upper right")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")

    plt.show()

In [ ]:
display_learning_curves(history)

In [ ]:
predicted_car = regressor_car.predict(x_test)

# Change Input For Other Models

In [ ]:
train_2_x = pd.read_csv("../input/tabular-playground-series-jul-2021/train.csv")
test_2_x = pd.read_csv("../input/tabular-playground-series-jul-2021/test.csv")
train_2_x = train_2_x.drop(["target_carbon_monoxide","target_benzene","target_nitrogen_oxides","date_time"],axis = 1)
test_2_x = test_2_x.drop(["date_time"],axis = 1)

scaler2 = StandardScaler()
train_2_x = scaler2.fit_transform(train_2_x)
test_2_x = scaler2.transform(test_2_x)

In [ ]:
"""regressor_benz = Sequential()
regressor_benz.add(Dense(25, input_dim=8, kernel_initializer='normal', activation='relu'))
regressor_benz.add(Dense(65, activation='relu'))
regressor_benz.add(Dropout(0.1))
regressor_benz.add(Dense(40, activation='relu'))
regressor_benz.add(Dropout(0.1))
regressor_benz.add(Dense(20, activation='relu'))
regressor_benz.add(Dense(8, activation='relu'))
regressor_benz.add(Dense(1, activation='linear'))
regressor_benz.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])
regressor_benz.fit(train_2_x, y_benz, epochs = 120, batch_size = 32)
predicted_benz = regressor_benz.predict(test_2_x)
"""


# Gradient Boost For Other Outputs

In [ ]:
gb_model_benz = GradientBoostingRegressor(subsample=0.7,learning_rate=0.017, max_depth=4 ,
                                          random_state=np.random.randint(1000),n_estimators=800).fit(train_2_x, y_benz)


print("Performance on train data:", gb_model.score(train_2_x, y_benz))

In [ ]:
predicted_benz  = gb_model_benz.predict(test_2_x)

In [ ]:
regressor_nit.fit(x, y_nit, epochs = 100, batch_size = 32)

In [ ]:
"""dtr_nit = DecisionTreeRegressor()
print("Performance on train data:", gb_model.score(train_2_x, y_benz))"""

In [ ]:
xgb_model_nit = XGBRegressor(subsample=0.7, learning_rate=0.02, max_depth=4,
                             random_state=np.random.randint(1000),n_estimators=800).fit(train_2_x, y_nit)

print("Performance on train data:", xgb_model_nit.score(train_2_x, y_nit))

In [ ]:
gb_model_nit = GradientBoostingRegressor(subsample=0.7,learning_rate=0.017, max_depth=4 
                                         ,random_state=np.random.randint(1000),n_estimators=800).fit(train_2_x, y_nit)

print("Performance on train data:", gb_model_nit.score(train_2_x, y_nit))

In [ ]:
predicted_nit = xgb_model.predict(test_2_x)

# Merge Output

In [ ]:
predicted_car_df = pd.DataFrame(predicted_car)
predicted_benz_df = pd.DataFrame(predicted_benz)
predicted_nit_df = pd.DataFrame(predicted_nit)

In [ ]:
output = pd.concat([pred_times, predicted_car_df,predicted_benz_df,predicted_nit_df], axis=1 ,ignore_index=True)

In [ ]:
output=output.rename(columns={0: 'date_time', 1: 'target_carbon_monoxide' , 2: 'target_benzene', 3: 'target_nitrogen_oxides'})

In [ ]:
output

In [ ]:
output.to_csv("leak_output.csv", index = False)